In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("longshort_strategy.ipynb")

In [2]:
import datetime as dt
import pandas as pd
import numpy as np
import warnings
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import zscore
from sklearn.metrics import mean_squared_error, r2_score

warnings.filterwarnings('ignore')

In [3]:
def isclose(value, original, tolerance= 0.05):
    return value <= original * (1+tolerance) and value >= original * (1-tolerance)

In this homework, we will:
- Introduce pandas through data cleaning and processing.
- Implement a basic trading strategy.
- Model the costs of a trading strategy.
- Calculate expected returns.
- Use expected returns to create a trading strategy.
- Try to add predictive features/signals.

# Pandas intro

In [ ]:
!pip install lxml

In [6]:
# list of sp500 companies, from wikipedia
url = 'https://en.m.wikipedia.org/w/index.php?title=List_of_S%26P_500_companies&oldid=1219339331'

# pandas lets us read html tables from wikipedia page
data = pd.read_html(url)

You can add cells in the notebook to perform quick operations or tests. Note that the last value in a cell will automatically be displayed below the cell once it finishes running. Let's examine the type of variable that `data` returns.

In [ ]:
type(data)

We can access values in a list using indexing and slicing operations:

- `list[i]` will return the \(i\)-th value in the list, starting with 0.
- `list[i:]` will return the \(i\)-th value and onward.
- `list[:i]` will return all values up to the \(i\)-th value.
- `list[i:j]` will return values from the \(i\)-th value onward, up to but not including the \(j\)-th value.

In [ ]:
# looks like data returns a list of tables, what type is each table? 
type(data[0])

In [11]:
# the dataframe is the core object of pandas, we'll abbreviate as df
# a dataframe is essentially a code version of a table
df = data[0]
df

In [15]:
# we can look at the first n rows of a table using df.head(n)
# please do so here
df_head5 = ...

In [16]:
grader.check("q1a")

In this analysis, we are not particularly interested in the columns for security, headquarters, date added, founded, or CIK values. We can select specific columns of a DataFrame by passing a list of column names:

```python
columns = ['colname1', 'colname2', ...]
df[columns]
```

Please select every column except for security, headquarters, CIK, founded, and date added.

In [ ]:
columns = 

In [ ]:
grader.check("q1b")

It appears there are quite a few sectors! In a DataFrame, one column is referred to as a series. We can select a series using `df['colname']`. By using `series.unique()`, we can obtain a list of the unique values within a series.

Please return the number of unique values in the 'GICS Sector' column. Note that you can programmatically access the length of this array using `.size`.

In [ ]:
unique_sectors = ...

In [ ]:
grader.check("q1c")

If we want to determine how many times each unique value appears in a series, we can use `series.value_counts()`. This function associates each unique value with the number of times it appears in the series.

Please return the number of times 'Information Technology' appears in the 'GICS Sector' column. Note that you can programmatically access the count for 'Information Technology' using `series.value_counts()['uniquevalname']`.

In [ ]:
it_sectors = ...

In [ ]:
grader.check("q1d")

One way to select values from a DataFrame is by using index-based location, or `iloc`. We can use `df.iloc[index for rows, index for columns]` to specify which indices to select.

- `df.iloc[:,:]` will return the entire DataFrame.
- `df.iloc[:5, :6]` will return the first 5 rows and 6 columns.
- `df.iloc[2:5, 3:]` will return the third column and onward, for rows 2-4.

Please return rows 5-9, and return the 2nd column and onward.

In [ ]:
specified_slice = ...

In [ ]:
grader.check("q1e")

We can also examine the sub-industries, each of which is associated with one of the sectors. To select only the rows of the DataFrame that correspond to a specific sector, we can use boolean indexing.

A boolean operator can be created generally using `df['colname'] {comparator} {value}`. This returns a series of `False` and `True` values for each row.

Please create a boolean operator for when the 'GICS Sector' is 'Health Care' and return the number of `False` values using `value_counts()`.

In [ ]:
false_healthcare = ...

In [ ]:
grader.check("q1f")

Now that we have our boolean operator, we can select the values that are `True`. We do this using `df[operator]`, where `operator = df['colname'] {comparator} {value}`.

Please use the operator you created to select only the rows where the security is in the 'Energy' sector (using the 'GICS Sector' column).

In [ ]:
energy = ...

In [ ]:
grader.check("q1g")

We also have `df.loc`, which works similarly to `df.iloc`. Here are some examples:

- `df.loc[row_indexer, column_indexer]`
- `df.loc[start_row_label:end_row_label, start_col_label:end_col_label]`
- `df.loc[list_of_row_labels, list_of_column_labels]`
- `df.loc[boolean_array_for_rows, boolean_array_for_columns]`

Please select the rows where the 'GICS Sector' is 'Industrials', and return the 1st through 4th columns.

In [ ]:
first4cols_industrials = ...

In [ ]:
grader.check("q1h")

Another useful DataFrame operation is sorting. We can sort an entire DataFrame using `df.sort_values('colname', ascending=True/False)`.

Please sort, in ascending order, the symbols in the 'Consumer Discretionary' sector, and report the 4th oldest security (by date added). You may want to break this into three separate tasks to make it easier.

In [ ]:
old_4_security = ...

In [ ]:
grader.check("q1i")

Let's also introduce the `groupby` operator. We can group a DataFrame based on a given column, such as the 'GICS Sector' column. After grouping a DataFrame, we commonly apply some kind of aggregation, such as a count, an average, etc. For example, we might use `df.groupby('colname').count()`.

Please return the average CIK number of the 'Real Estate' sector using the `df.groupby('colname').mean(numeric_only=True)` aggregation. You can also access a specific series by adding `df.groupby('colname1').mean(numeric_only=True)['colname2']` to view the aggregation of a particular series.

In [ ]:
mean_realestate_cik = ...

In [ ]:
grader.check("q1j")

In [ ]:
def get_common_subindustry(df):
    # .index returns the index of a series, in this case the unique values
    # since value counts is sorted in descending order
    # .value_counts().index[0] will give us the largest avlue
    
    # we can also use display to show the intermediate steps
    # we use series.values to access the list of values in the series
    
    print('-'*35)
    print(df['GICS Sector'].values[0])
    print('-'*35)
    display(df['GICS Sub-Industry'].value_counts())
    return df['GICS Sub-Industry'].value_counts().index[0]

In [ ]:
df['GICS Sub-Industry'].value_counts().index[0]

In [ ]:
df.groupby('GICS Sector').apply(get_common_subindustry)

# data downloading/cleaning

In [ ]:
# the yfinance library provides access to lots of information about tickers available on yahoo finance
!pip install yfinance

In [13]:
import yfinance as yf

**Errors are expected as data is missing for some tickers, see the future cells!**

In [ ]:
# we can download historical price data for these assets using
# yf.download(tickers)
# there's other optional parameters that we won't cover here
# lets get the list of symbols/tickers we care about, and then download them using .unique() on our dataframe
# convert the series to a list before you pass it to yf
# this may take a minute or two
# please put the data into a dataframe called hdf, or historical dataframe for short

tickers = list(df['Symbol'].unique())
hdf = yf.download(tickers, start='2023-10-01', end='2024-03-29')
hdf = hdf[hdf.index <= '2024-03-19']

In [ ]:
grader.check("q2a")

Let's take a look at the data we've just downloaded

In [24]:
hdf

In [25]:
# looks like the index is the date, and we have multiple levels of columns!
# we can get the top level of columns using df.columns.get_level_values(i)
# let's look at the unique top level columns, with .unique()

In [ ]:
hdf.columns.get_level_values(0).unique()

In [27]:
# the first level of columns has the 'adjusted close', or the closing price for that day, with interest rate adjusted
# we are also given 'volume', or the amount of shares traded that day
# the other stuff might be nice to have, but we'll just stick with adjusted close and volume for simplicity
# let's now select those
# to not destroy our old dataframe, we'll make a copy called hdf_raw with df.copy()
# once you make the copy, make hdf = hdf[columns]
hdf_raw = hdf.copy()
hdf = hdf[['Adj Close', 'Volume']]

2b or not 2b, the answer is not because I deleted the question :) Also, **the next cell is filled in for you, so there's nothing you need to do here.**

In [29]:
# you may have seen the following error message upon download: 

# 2 Failed downloads:
# - BF.B: No data found for this date range, symbol may be delisted
# - BRK.B: No data found, symbol may be delisted
# - GEV and SOLV are also recent additions without any data

# please use df.loc to view the data for these symbols
# you should return all rows, and use columns.get_level_values(1) to get the second level of columns, with the symbols
# note that you can use '.isin(list)' as a boolean operator to return if a value is in a list
# your code might look like

# df.loc[:,df.columns.get_level_values(1).isin([list of values to check])]
broken_df = hdf.loc[:,hdf.columns.get_level_values(1).isin(['BRK.B', 'BF.B', 'GEV', 'SOLV', 'PXD', 'WRK', 'CTLT', 'MRO'])]
broken_df

In [ ]:
grader.check("q2c")

In [ ]:
# awesome, now lets select everything except for that, since we dont want to use this data
# we can invert a boolean indexer by using ~, eg. df.loc[:, ~<condition>]
# let hdf be equal to this new selection, and then drop nan values using df.dropna()
hdf = 
hdf = 

In [ ]:
grader.check("q2d")

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Please also check gradescope for any written assignments for this week.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)